In [2]:
pip install optuna

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost --upgrade

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 24.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.1
    Uninstalling xgboost-2.1.1:
      Successfully uninstalled xgboost-2.1.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np

# Load your dataset
data = pd.read_csv('Medicalpremium.csv')

# Define features and target variable
X = data.drop('PremiumPrice', axis=1)  # Features (all columns except target)
y = data['PremiumPrice']                # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    model_name = trial.suggest_categorical('model', ['Random Forest', 'XGBoost', 'Linear Regression', 'SVM', 'Gradient Boosting'])

    if model_name == 'Random Forest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 20)
        }
        model = RandomForestRegressor(**params)

    elif model_name == 'XGBoost':
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5)
        }
        model = XGBRegressor(**params)

    elif model_name == 'SVM':
        params = {
            'kernel': trial.suggest_categorical('svm_kernel', ['linear', 'rbf', 'poly']),
            'C': trial.suggest_float('svm_C', 0.1, 10.0)
        }
        model = SVR(**params)

    elif model_name == 'Gradient Boosting':
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
        }
        model = GradientBoostingRegressor(**params)

    else:  # Linear Regression
        model = LinearRegression()

    # Fit the model and make predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # MAPE as percentage

    return r2  # Return R-squared value for optimization

# Create the Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the results
print("\nModel Performance Metrics-----")

# Evaluate and print results for each model
models = ['Random Forest', 'SVM', 'XGBoost', 'Linear Regression', 'Gradient Boosting']
for model_name in models:
    print(f"\nEvaluating {model_name}...")

    # Initialize and fit the model with the best parameters
    if model_name == 'Random Forest':
        best_model = RandomForestRegressor(n_estimators=study.best_params.get('n_estimators', 100),
                                            max_depth=study.best_params.get('max_depth', 10))
    elif model_name == 'XGBoost':
        best_model = XGBRegressor(
            learning_rate=study.best_params.get('learning_rate', 0.1),
            max_depth=study.best_params.get('max_depth', 3),
            n_estimators=study.best_params.get('n_estimators', 100),
            min_child_weight=study.best_params.get('min_child_weight', 1),
            subsample=study.best_params.get('subsample', 1.0),
            gamma=study.best_params.get('gamma', 0)
        )
    elif model_name == 'SVM':
        best_model = SVR(
            kernel=study.best_params.get('svm_kernel', 'linear'),
            C=study.best_params.get('svm_C', 1.0)
        )
    elif model_name == 'Gradient Boosting':
        best_model = GradientBoostingRegressor(
            learning_rate=study.best_params.get('learning_rate', 0.1),
            n_estimators=study.best_params.get('n_estimators', 100),
            max_depth=study.best_params.get('max_depth', 3),
            min_samples_split=study.best_params.get('min_samples_split', 2)
        )
    else:  # Linear Regression
        best_model = LinearRegression()

    # Fit the model and make predictions
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # MAPE as percentage

    # Print metrics
    print(f"{model_name} MSE: {mse:.4f}")
    print(f"{model_name} RMSE: {rmse:.4f}")
    print(f"{model_name} R-squared: {r2:.4f}")
    print(f"{model_name} MAPE: {mape:.2f}%")


/home/talisman/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-10 11:47:52,770] A new study created in memory with name: no-name-a1d99f7c-96d9-4b30-8010-00ae0d3e87d9
[I 2024-11-10 11:47:53,555] Trial 0 finished with value: 0.8135449711971112 and parameters: {'model': 'Gradient Boosting', 'learning_rate': 0.03989183927265595, 'n_estimators': 489, 'max_depth': 5, 'min_samples_split': 3}. Best is trial 0 with value: 0.8135449711971112.
[I 2024-11-10 11:47:53,939] Trial 1 finished with value: 0.7700737115190077 and parameters: {'model': 'Gradient Boosting', 'learning_rate': 0.18754175947476098, 'n_estimators': 155, 'max_depth': 10, 'min_samples_split': 6}. Best is trial 0 with value: 0.8135449711971112.
[I 2024-11-10 11:47:54,790] Trial 2 finished with value: 0.8350210785865784 and paramet


Model Performance Metrics-----

Evaluating Random Forest...
Random Forest MSE: 5119930.0291
Random Forest RMSE: 2262.7262
Random Forest R-squared: 0.8799
Random Forest MAPE: 4.47%

Evaluating SVM...
SVM MSE: 20710921.6290
SVM RMSE: 4550.9254
SVM R-squared: 0.5143
SVM MAPE: 12.04%

Evaluating XGBoost...
XGBoost MSE: 8478340.8757
XGBoost RMSE: 2911.7591
XGBoost R-squared: 0.8012
XGBoost MAPE: 4.72%

Evaluating Linear Regression...
Linear Regression MSE: 12221661.7059
Linear Regression RMSE: 3495.9493
Linear Regression R-squared: 0.7134
Linear Regression MAPE: 10.96%

Evaluating Gradient Boosting...
Gradient Boosting MSE: 14074829.2345
Gradient Boosting RMSE: 3751.6435
Gradient Boosting R-squared: 0.6699
Gradient Boosting MAPE: 5.22%
